In [56]:
import tensorflow as tf
from pathlib import Path
import os

In [57]:
%pwd

'c:\\Users\\Vador66\\Project\\Chest_Cancer'

In [6]:
os.chdir('../')

In [65]:
#entity
from dataclasses import dataclass
from pathlib import Path
from Chest_Cancer_Classifier.config.configuration import TrainingConfig

@dataclass(frozen=True)
class ExtendedTrainingConfig(TrainingConfig):
    checkpoint_model_train : Path

In [66]:
from Chest_Cancer_Classifier.constants import *
from Chest_Cancer_Classifier.utils.common import read_yaml, create_directories

In [84]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> ExtendedTrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = ExtendedTrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            checkpoint_model_train= Path("artifacts/prepare_base_model/train03.h5"),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [85]:
# Assurez-vous que la classe ConfigurationManager est définie comme indiqué précédemment

# Créez une instance de ConfigurationManager
config_manager = ConfigurationManager()

# Appelez la méthode get_training_config pour obtenir l'objet training_config
training_config = config_manager.get_training_config()

# Affichez l'objet training_config pour vérifier son contenu
print(training_config)

[2024-08-29 02:34:00,967: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-29 02:34:00,969: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-29 02:34:00,970: INFO: common: created directory at: artifacts]
[2024-08-29 02:34:00,971: INFO: common: created directory at: artifacts\training]
ExtendedTrainingConfig(root_dir=WindowsPath('artifacts/training'), trained_model_path=WindowsPath('artifacts/training/model.h5'), updated_base_model_path=WindowsPath('artifacts/prepare_base_model/base_model_updated.h5'), training_data=WindowsPath('artifacts/data_ingestion/Data'), params_is_augmentation=True, params_image_size=BoxList([224, 224, 3]), params_batch_size=8, params_epochs=15, checkpoint_model_train=WindowsPath('artifacts/prepare_base_model/train03.h5'))


In [86]:
from keras.preprocessing.image import ImageDataGenerator

In [87]:
class Training2:
    def __init__(self, config:ExtendedTrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.checkpoint_model_train)
        self.model._name = "trained_04"
        self.model.trainable = True
    

    def print_model_summary(self):
        if hasattr(self, 'model'):
            self.model.summary(show_trainable=True)
        else:
            print("Model is not loaded yet.")

    def train_valid_generator(self):

        train_datagen = ImageDataGenerator(
            rescale = 1./255,                    
            rotation_range=5,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            #zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest')

        valid_datagen = ImageDataGenerator(rescale = 1./255,
                                        validation_split = 0.2)

        test_datagen  = ImageDataGenerator(rescale = 1./255)

        self.train_dataset  = train_datagen.flow_from_directory(
                    directory = os.path.join(self.config.training_data,'train'),
                    target_size = (224,224),
                    class_mode = 'categorical',
                    batch_size = self.config.params_batch_size
                    )
        self.val_dataset  = valid_datagen.flow_from_directory(
                    directory = os.path.join(self.config.training_data,'valid'),
                    target_size = (224,224),
                    class_mode = 'categorical',
                    batch_size = self.config.params_batch_size
                    )
        self.test_dataset  = test_datagen.flow_from_directory(
                    directory = os.path.join(self.config.training_data,'test'),
                    target_size = (224,224),
                    class_mode = 'categorical',
                    batch_size = self.config.params_batch_size
                    )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.train_dataset.samples // self.train_dataset.batch_size
        self.validation_steps = self.val_dataset.samples // self.val_dataset.batch_size

        self.model.fit(
            self.train_dataset,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.val_dataset
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [88]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training2(config=training_config)
    training.get_base_model()
    training.print_model_summary()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-08-29 02:34:08,780: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-29 02:34:08,782: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-29 02:34:08,783: INFO: common: created directory at: artifacts]
[2024-08-29 02:34:08,785: INFO: common: created directory at: artifacts\training]
Model: "trained_04"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      Y          
                                                                            
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     Y          
                                                                            
 block1_pool (MaxPooling2D)  (None

In [89]:
import dagshub
dagshub.init(repo_owner='MolAlexandre', repo_name='Chest_Cancer', mlflow=True)

model = tf.keras.models.load_model("artifacts/training/model.h5")


Initialized MLflow to track repo "MolAlexandre/Chest_Cancer"

[2024-08-29 02:55:21,183: INFO: helpers: Initialized MLflow to track repo "MolAlexandre/Chest_Cancer"]


Repository MolAlexandre/Chest_Cancer initialized!

[2024-08-29 02:55:21,190: INFO: helpers: Repository MolAlexandre/Chest_Cancer initialized!]
[2024-08-29 02:55:21,637: WARNING: hdf5_format: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.]


In [90]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [91]:
from Chest_Cancer_Classifier.constants import *
from Chest_Cancer_Classifier.utils.common import read_yaml, create_directories, save_json

In [92]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Data",
            mlflow_uri="https://dagshub.com/MolAlexandre/Chest_Cancer.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [93]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from mlflow.models import infer_signature

from keras.preprocessing.image import ImageDataGenerator

In [94]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)
        
        self.val_dataset  = valid_datagen.flow_from_directory(
                    directory = os.path.join(self.config.training_data,'valid'),
                    target_size = self.config.params_image_size[:-1],
                    class_mode = 'categorical',
                    batch_size = self.config.params_batch_size
                    )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.val_dataset)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # Set a tag that we can use to remind ourselves what this run was for
            mlflow.set_tag("Training Info", "Cancer retrain unfreeze")

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [95]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-29 02:55:51,489: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-29 02:55:51,492: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-29 02:55:51,492: INFO: common: created directory at: artifacts]
[2024-08-29 02:55:51,936: WARNING: hdf5_format: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.]
Found 72 images belonging to 4 classes.
9/9 [==============================] - 8s 883ms/step - loss: 1.0185 - accuracy: 0.5000
[2024-08-29 02:56:00,249: INFO: common: json file saved at: scores.json]


2024/08/29 02:56:01 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-08-29 02:56:02,915: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Vador66\AppData\Local\Temp\tmpowkmob_l\model\data\model\assets
[2024-08-29 02:56:04,115: INFO: builder_impl: Assets written to: C:\Users\Vador66\AppData\Local\Temp\tmpowkmob_l\model\data\model\assets]


c:\Users\Vador66\Project\Chest_Cancer\cancer\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/08/29 02:56:30 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.
